In [1]:
from keras.applications.vgg19 import VGG19
from keras.applications.vgg19 import preprocess_input
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
from keras.models import Model
import numpy as np 
import pandas as pd 
from subprocess import check_output
#print(check_output(["ls", "../input/"]))
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error,mean_absolute_error,log_loss
from sklearn.preprocessing import PolynomialFeatures
from os.path import join as opj
from matplotlib import pyplot as plt
import tensorflow as tf
import os
%matplotlib inline

Using TensorFlow backend.


In [2]:
from keras.layers.advanced_activations import LeakyReLU, PReLU
from keras.applications.inception_v3 import InceptionV3
from keras.applications.xception import Xception
from keras.applications.mobilenet import MobileNet
from keras.layers import Concatenate, LSTM, concatenate
from matplotlib import pyplot
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Input, Flatten, Activation,Add, ZeroPadding2D ,AveragePooling2D
from keras.layers import GlobalMaxPooling2D, GlobalAveragePooling2D
from keras.layers.normalization import BatchNormalization
from keras.layers.merge import Concatenate
from keras.models import Model
from keras import initializers,regularizers
from keras.optimizers import Adam,SGD,RMSprop
from keras.initializers import glorot_uniform
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping
from keras.preprocessing.image import ImageDataGenerator
import keras.backend as K
K.set_image_data_format('channels_last')
#K.set_learning_phase(1)

In [3]:
train = pd.read_json("input/train.json")

### Fill Inc_angle

In [4]:
#angle_nona = train[train['inc_angle']!='na']
#angle_nona = angle_nona.reset_index()
only_band1 = ['band1_'+str(i) for i in range(75*75)]
only_band2 = ['band2_'+str(i) for i in range(75*75)]
both_bands = only_band1+only_band2
df = pd.concat([pd.DataFrame(train['band_1'].values.tolist(),columns = only_band1),
                pd.DataFrame(train['band_2'].values.tolist(),columns = only_band2)],axis = 1)
df['b1_median'] = np.median(df[only_band1],axis = 1)
df['b2_median'] = np.median(df[only_band2],axis = 1)
df['b1_mean'] = np.mean(df[only_band1],axis = 1)
df['b2_mean'] = np.mean(df[only_band2],axis = 1)
df['b1_max'] = np.max(df[only_band1],axis = 1)
df['b2_max'] = np.max(df[only_band2],axis = 1)
df['b1_min'] = np.median(df[only_band1],axis = 1)
df['b2_min'] = np.median(df[only_band2],axis = 1)
df['b1_std'] = np.std(df[only_band1],axis = 1)
df['b2_std'] = np.std(df[only_band2],axis = 1)
df['b1_var'] = np.var(df[only_band1],axis = 1)
df['b2_var'] = np.var(df[only_band2],axis = 1)
stats = df.columns[-12:]
df['inc_angle'] = train['inc_angle']
angle_na = df[df['inc_angle']=='na']
angle_nona = df[df['inc_angle']!='na']
X_train = angle_nona[stats]
X_pred = angle_na[stats]
y_train = angle_nona['inc_angle']
lr_model = LinearRegression().fit(X_train,y_train)
angle_pred = lr_model.predict(X_pred)
angle_na['inc_angle'] = angle_pred
new = pd.concat([angle_nona,angle_na],axis = 0).sort_index()
train = train.sort_index()
train['inc_angle'] = new['inc_angle']

c:\users\yltbe\anaconda3\envs\tensorflow_gpu\lib\site-packages\ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [9]:
train

,band_1,band_2,id,inc_angle,is_iceberg,b1_median,b2_median,b1_mean,b2_mean,b1_max,b2_max,b1_min,b2_min,b1_std,b2_std,b1_var,b2_var
0,"[-27.878360999999998, -27.15416, -28.668615, -...","[-27.154118, -29.537888, -31.0306, -32.190483,...",dfd5f913,43.9239,0,-27.879921,-30.007847,-27.911045,-29.910116,-0.213149,-11.252153,-38.211376,-41.135918,2.764537,2.381284,7.642667,5.670513
1,"[-12.242375, -14.920304999999999, -14.920363, ...","[-31.506321, -27.984554, -26.645678, -23.76760...",e25388fd,38.1562,0,-13.654199,-25.486130,-13.566553,-25.359106,12.570409,0.044052,-23.125309,-34.765831,3.142532,2.934098,9.875504,8.608931
2,"[-24.603676, -24.603714, -24.871029, -23.15277...","[-24.870956, -24.092632, -20.653963, -19.41104...",58b2aaa0,45.2859,1,-22.935019,-24.606205,-23.053699,-24.839823,-9.918477,-15.605879,-33.391197,-34.148819,2.223906,2.125275,4.945755,4.516793
3,"[-22.454607, -23.082819, -23.998013, -23.99805...","[-27.889421, -27.519794, -27.165262, -29.10350...",4cfc3a18,43.8306,0,-23.303238,-29.549923,-23.210772,-29.567911,4.795627,-5.554516,-32.204136,-39.564053,2.566233,2.389980,6.585550,5.712003
4,"[-26.006956, -23.164886, -23.164886, -26.89116...","[-27.206915, -30.259186, -30.259186, -23.16495...",271f93f4,35.6256,0,-25.206615,-25.466921,-25.280027,-25.627728,-6.956036,-9.434006,-35.010487,-40.276115,2.305288,2.276244,5.314351,5.181286
5,"[-20.769371, -20.769434, -25.906025, -25.90602...","[-29.288746, -29.712593, -28.884804, -28.88480...",b51d18b5,36.9034,1,-20.317537,-27.110518,-19.999609,-27.098206,3.864040,-9.127563,-29.715744,-37.674038,3.324674,2.770907,11.053460,7.677924
6,"[-26.673811, -23.666162, -27.622442, -28.31768...","[-24.557735, -26.97868, -27.622442, -29.073456...",31da1a04,34.4751,1,-25.294981,-26.383532,-25.180380,-26.566231,-10.350115,-17.439672,-35.926098,-36.842224,2.654438,2.143797,7.046043,4.595865
7,"[-24.989119, -27.755224, -25.817074, -24.98927...","[-27.755173, -26.732174, -28.124943, -31.83772...",56929c16,41.1769,0,-26.735996,-27.758291,-26.969431,-27.980122,-13.997642,-22.108116,-37.300262,-37.297852,2.412018,2.114239,5.817832,4.470006
8,"[-17.146641, -17.146572, -17.994583, -19.44553...","[-25.733608, -24.472507, -24.710424, -22.77215...",525ab75c,35.7829,0,-16.560719,-23.580057,-15.603797,-23.624271,28.578884,-1.074512,-25.732534,-34.251976,5.194528,2.693885,26.983122,7.257014
9,"[-24.020853, -23.551275, -27.18819, -29.126434...","[-28.702518, -33.563324, -29.571918, -29.12643...",192f56eb,43.3007,0,-25.047281,-28.704334,-25.289223,-28.826078,-5.606295,-15.594248,-36.065300,-38.248188,2.334278,2.192880,5.448853,4.808722


In [6]:
X_angle = np.array(train['inc_angle'])

In [3]:
test = pd.read_json("input/test.json")

In [7]:
X_band_1=np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in train["band_1"]])
X_band_2=np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in train["band_2"]])
X_train = np.concatenate([X_band_1[:, :, :, np.newaxis], X_band_2[:, :, :, np.newaxis],((X_band_1+X_band_2)/2)[:, :, :, np.newaxis]], axis=-1)
target_train=train['is_iceberg']

In [8]:
train['b1_median'] = [np.median(i) for i in X_band_1]
train['b2_median'] = [np.median(i) for i in X_band_2]
train['b1_mean'] = [np.mean(i) for i in X_band_1]
train['b2_mean'] = [np.mean(i) for i in X_band_2]
train['b1_max'] = [np.max(i) for i in X_band_1]
train['b2_max'] = [np.max(i) for i in X_band_2]
train['b1_min'] = [np.min(i) for i in X_band_1]
train['b2_min'] = [np.min(i) for i in X_band_2]
train['b1_std'] = [np.std(i) for i in X_band_1]
train['b2_std'] = [np.std(i) for i in X_band_2]
train['b1_var'] = [np.var(i) for i in X_band_1]
train['b2_var'] = [np.var(i) for i in X_band_2]

In [9]:
X_other = train.drop(['band_1','band_2','id','is_iceberg'],axis = 1)

In [10]:
X_train_cv, X_valid, X_train_other, X_valid_other , y_train_cv, y_valid = train_test_split(X_train,X_other, target_train, random_state=2,stratify = target_train, train_size=0.80)

c:\users\yltbe\anaconda3\envs\tensorflow_gpu\lib\site-packages\sklearn\model_selection\_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


### Normal Data Augmentation

In [11]:
def data_augmentation(X_input,Y_input,batch_size = 32):
    data_aug = ImageDataGenerator(#featurewise_center=True,
                             #featurewise_std_normalization=True,
                             #zca_whitening=True,
                             rotation_range=20,
                             width_shift_range = 0.2,
                             height_shift_range = 0.2,
                             zoom_range = 0.2,
                             data_format = 'channels_last',
                             horizontal_flip = True,
                             vertical_flip = True)#,fill_mode = 'constant',cval = 0)
    data_aug_batches = data_aug.flow(X_input,Y_input,batch_size = batch_size)
    return data_aug_batches

### Data Augmentation with Inc_angle

In [12]:
gen = ImageDataGenerator(#featurewise_center=True,
                             #featurewise_std_normalization=True,
                             #zca_whitening=True,
                             rotation_range=40,
                             width_shift_range = 0,
                             height_shift_range = 0,
                             zoom_range = 0.2,
                             data_format = 'channels_last',
                             horizontal_flip = True,
                             vertical_flip = True,fill_mode = 'constant',cval = 0)

def gen_with_angle(X1, X2, y,batch_size = 32):
    genX1 = gen.flow(X1,y,  batch_size=batch_size,seed = 1204)
    genX2 = gen.flow(X1,X2, batch_size=batch_size,seed = 1204)
    while True:
            X1i = genX1.next()
            X2i = genX2.next()
            #Assert arrays are equal - this was for peace of mind, but slows down training
            #np.testing.assert_array_equal(X1i[0],X2i[0])
            yield [X1i[0], X2i[1]], X1i[1]

### VGG19

In [13]:

def vggmodel(lr = 0.0001):
    input_other = Input(shape=(11,), name="angle")
    #First composite layer
    base_model = VGG16(include_top=False,weights='imagenet',input_shape=(75,75,3),classes=1)
    for layer in base_model.layers[:7]:
        layer.trainable = False
    base_model2 = VGG19(include_top=False,weights = 'imagenet',input_tensor = base_model.input,input_shape = (75,75,3),classes = 1)
    for layer in base_model2.layers[:7]:
        layer.trainable = False
    for layer in base_model2.layers:
        layer.name = layer.name+'(2)'
    Y = base_model2.output
    W = base_model.output
    W = GlobalMaxPooling2D()(W)
    Y = GlobalMaxPooling2D()(Y)
    W = concatenate([W, input_other])
    W = Dense(1024,activation= 'relu')(W)
    W = Dropout(0.3)(W)
    W = Dense(1024,activation= 'relu')(W)
    W = Dropout(0.3)(W)
    Y = concatenate([Y, input_other])
    Y = Dense(1024,activation= 'relu')(Y)
    Y = Dropout(0.3)(Y)
    Y = Dense(1024,activation= 'relu')(Y)
    Y = Dropout(0.3)(Y)
    #X = Dense(256,activation= 'relu')(X)
    #X = BatchNormalization()(X)
    #X = Dropout(0.2)(X)
    #Decision layer
    X = concatenate([W,Y])
    X = Dense(1024,activation= 'relu')(X)
    X = Dense(1,activation = 'sigmoid')(X)
    
    model = Model(inputs=[base_model.input,input_other],outputs=X)
    optimizer=Adam(lr, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
    #optimizer = SGD(lr, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(loss='binary_crossentropy',
                  optimizer=optimizer,
                  metrics=['accuracy'])
    model.summary()
    return model

def get_callbacks(filepath = ".model_weights.hdf5", patience=7):
    #es = EarlyStopping('val_loss', patience=patience, mode="min")
    msave = ModelCheckpoint(filepath, save_best_only=True)
    return [msave]

#callbacks = get_callbacks(filepath=file_path, patience=3)

In [14]:
Vggmodel=vggmodel(lr = 0.0001)
def fitmodel(model,X_train,y_train,X_valid,y_valid,augment = False,with_angle = False,epochs = 50,batch_size = 32,filepath = ".model_weights.hdf5"):
    if augment == False :
        result = model.fit(X_train, y_train,batch_size=batch_size,epochs=epochs,verbose=1,shuffle = True,
                           validation_data=(X_valid, y_valid),callbacks=get_callbacks(filepath))
    else:
        if with_angle==False:
            result = model.fit_generator(data_augmentation(X_train,y_train,batch_size = batch_size),
                                         steps_per_epoch = len(X_train_cv)/batch_size,epochs=epochs,verbose=1,shuffle = True,
                                         validation_data=(X_valid, y_valid),callbacks=get_callbacks(filepath))
        else:
            result = model.fit_generator(gen_with_angle(X_train,X_train_other,y_train,batch_size = batch_size),
                                         steps_per_epoch = len(X_train_cv)/batch_size,epochs=epochs,verbose=1,shuffle = True,
                                         validation_data=([X_valid,X_valid_other], y_valid),callbacks=get_callbacks(filepath))
    return result

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1(2) (InputLayer)         (None, 75, 75, 3)    0                                            
__________________________________________________________________________________________________
block1_conv1(2) (Conv2D)        (None, 75, 75, 64)   1792        input_1(2)[0][0]                 
__________________________________________________________________________________________________
block1_conv2(2) (Conv2D)        (None, 75, 75, 64)   36928       block1_conv1(2)[0][0]            
__________________________________________________________________________________________________
block1_pool(2) (MaxPooling2D)   (None, 37, 37, 64)   0           block1_conv2(2)[0][0]            
__________________________________________________________________________________________________
block1_con

In [15]:
result_Vgg = fitmodel(Vggmodel,X_train_cv,y_train_cv,X_valid,y_valid,augment = True,with_angle=True,epochs = 100,batch_size = 32,filepath = 'model save/Model 4-VGG16 with predicted inc_angle/first.hdf5')

Epoch 1/100
41/40 [==============================] - 68s 2s/step - loss: 0.8140 - acc: 0.6107 - val_loss: 0.6380 - val_acc: 0.6168
Epoch 2/100
41/40 [==============================] - 59s 1s/step - loss: 0.5877 - acc: 0.7155 - val_loss: 0.3658 - val_acc: 0.8442
Epoch 3/100
41/40 [==============================] - 55s 1s/step - loss: 0.5308 - acc: 0.7732 - val_loss: 0.3849 - val_acc: 0.8193
Epoch 4/100
41/40 [==============================] - 55s 1s/step - loss: 0.5072 - acc: 0.7694 - val_loss: 0.4420 - val_acc: 0.7850
Epoch 5/100
41/40 [==============================] - 55s 1s/step - loss: 0.4338 - acc: 0.8132 - val_loss: 0.4280 - val_acc: 0.8287
Epoch 6/100
41/40 [==============================] - 55s 1s/step - loss: 0.3963 - acc: 0.8383 - val_loss: 0.4031 - val_acc: 0.8287
Epoch 7/100
41/40 [==============================] - 59s 1s/step - loss: 0.3909 - acc: 0.8334 - val_loss: 0.3603 - val_acc: 0.8287
Epoch 8/100
41/40 [==============================] - 55s 1s/step - loss: 0.3410 - a

In [ ]:
# vgg = VGG19(weights='imagenet')
# model = Model(inputs=base_model.input, outputs=base_model.get_layer('block4_pool').output)

# img_path = 'elephant.jpg'
# img = image.load_img(img_path, target_size=(224, 224))
# x = image.img_to_array(img)
# x = np.expand_dims(x, axis=0)
# x = preprocess_input(x)

# block4_pool_features = model.predict(x)

In [7]:
target_train=train['is_iceberg']
test['inc_angle']=pd.to_numeric(test['inc_angle'], errors='coerce')
train['inc_angle']=pd.to_numeric(train['inc_angle'], errors='coerce')#We have only 133 NAs.
train['inc_angle']=train['inc_angle'].fillna(method='pad')
X_angle=train['inc_angle']
test['inc_angle']=pd.to_numeric(test['inc_angle'], errors='coerce')
X_test_angle=test['inc_angle']

#Generate the training data
X_band_1=np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in train["band_1"]])
X_band_2=np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in train["band_2"]])
X_band_3=(X_band_1+X_band_2)/2
#X_band_3=np.array([np.full((75, 75), angel).astype(np.float32) for angel in train["inc_angle"]])
X_train = np.concatenate([X_band_1[:, :, :, np.newaxis]
                          , X_band_2[:, :, :, np.newaxis]
                         , X_band_3[:, :, :, np.newaxis]], axis=-1)



X_band_test_1=np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in test["band_1"]])
X_band_test_2=np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in test["band_2"]])
X_band_test_3=(X_band_test_1+X_band_test_2)/2
#X_band_test_3=np.array([np.full((75, 75), angel).astype(np.float32) for angel in test["inc_angle"]])
X_test = np.concatenate([X_band_test_1[:, :, :, np.newaxis]
                          , X_band_test_2[:, :, :, np.newaxis]
                         , X_band_test_3[:, :, :, np.newaxis]], axis=-1)



In [13]:
from keras.preprocessing.image import ImageDataGenerator
batch_size=64
# Define the image transformations here
gen = ImageDataGenerator(horizontal_flip = True,
                         vertical_flip = True,
                         width_shift_range = 0.,
                         height_shift_range = 0.,
                         channel_shift_range=0,
                         zoom_range = 0.2,
                         rotation_range = 10)

# Here is the function that merges our two generators
# We use the exact same generator with the same random seed for both the y and angle arrays
def gen_flow_for_two_inputs(X1, X2, y):
    genX1 = gen.flow(X1,y,  batch_size=batch_size)
    genX2 = gen.flow(X1,X2, batch_size=batch_size)
    while True:
            X1i = genX1.next()
            X2i = genX2.next()
            #Assert arrays are equal - this was for peace of mind, but slows down training
            #np.testing.assert_array_equal(X1i[0],X2i[0])
            yield [X1i[0], X2i[1]], X1i[1]

# Finally create generator
def get_callbacks(filepath, patience=2):
    es = EarlyStopping('val_loss', patience=10, mode="min")
    msave = ModelCheckpoint(filepath, save_best_only=True)
    return [es, msave]


def getVggAngleModel():
    input_2 = Input(shape=[1], name="angle")
    angle_layer = Dense(1, )(input_2)
    base_model = VGG16(weights='imagenet', include_top=False, 
                 input_shape=X_train.shape[1:], classes=1)
    x = base_model.get_layer('block5_pool').output
    

    x = GlobalMaxPooling2D()(x)
    merge_one = concatenate([x, angle_layer])
    merge_one = Dense(512, activation='relu', name='fc2')(merge_one)
    merge_one = Dropout(0.3)(merge_one)
    merge_one = Dense(512, activation='relu', name='fc3')(merge_one)
    merge_one = Dropout(0.3)(merge_one)
    
    predictions = Dense(1, activation='sigmoid')(merge_one)
    
    model = Model(input=[base_model.input, input_2], output=predictions)
    
    sgd = SGD(lr=1e-3, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(loss='binary_crossentropy',
                  optimizer=sgd,
                  metrics=['accuracy'])
    return model


#Using K-fold Cross Validation with Data Augmentation.
def myAngleCV(X_train, X_angle, X_test):
    K=3
    folds = list(StratifiedKFold(n_splits=K, shuffle=True).split(X_train, target_train))
    y_test_pred_log = 0
    y_train_pred_log=0
    y_valid_pred_log = 0.0*target_train
    for j, (train_idx, test_idx) in enumerate(folds):
        print('\n===================FOLD=',j)
        X_train_cv = X_train[train_idx]
        y_train_cv = target_train[train_idx]
        X_holdout = X_train[test_idx]
        Y_holdout= target_train[test_idx]
        
        #Angle
        X_angle_cv=X_angle[train_idx]
        X_angle_hold=X_angle[test_idx]

        #define file path and get callbacks
        file_path = "%s_aug_model_weights.hdf5"%j
        callbacks = get_callbacks(filepath=file_path, patience=5)
        gen_flow = gen_flow_for_two_inputs(X_train_cv, X_angle_cv, y_train_cv)
        galaxyModel= getVggAngleModel()
        galaxyModel.fit_generator(
                gen_flow,
                steps_per_epoch=24,
                epochs=100,
                shuffle=True,
                verbose=1,
                validation_data=([X_holdout,X_angle_hold], Y_holdout),
                callbacks=callbacks)

        #Getting the Best Model
        galaxyModel.load_weights(filepath=file_path)
        #Getting Training Score
        score = galaxyModel.evaluate([X_train_cv,X_angle_cv], y_train_cv, verbose=0)
        print('Train loss:', score[0])
        print('Train accuracy:', score[1])
        #Getting Test Score
        score = galaxyModel.evaluate([X_holdout,X_angle_hold], Y_holdout, verbose=0)
        print('Test loss:', score[0])
        print('Test accuracy:', score[1])

        #Getting validation Score.
        pred_valid=galaxyModel.predict([X_holdout,X_angle_hold])
        y_valid_pred_log[test_idx] = pred_valid.reshape(pred_valid.shape[0])

        #Getting Test Scores
        temp_test=galaxyModel.predict([X_test, X_test_angle])
        y_test_pred_log+=temp_test.reshape(temp_test.shape[0])

        #Getting Train Scores
        temp_train=galaxyModel.predict([X_train, X_angle])
        y_train_pred_log+=temp_train.reshape(temp_train.shape[0])

    y_test_pred_log=y_test_pred_log/K
    y_train_pred_log=y_train_pred_log/K

    print('\n Train Log Loss Validation= ',log_loss(target_train, y_train_pred_log))
    print(' Test Log Loss Validation= ',log_loss(target_train, y_valid_pred_log))
    return y_test_pred_log

In [14]:
preds=myAngleCV(X_train, X_angle, X_test)


===================FOLD= 0


c:\users\yltbe\anaconda3\envs\tensorflow_gpu\lib\site-packages\ipykernel_launcher.py:48: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=Tensor("de...)`


Epoch 1/100
24/24 [==============================] - 32s 1s/step - loss: 0.6953 - acc: 0.5833 - val_loss: 0.4926 - val_acc: 0.7495
Epoch 2/100
24/24 [==============================] - 29s 1s/step - loss: 0.5155 - acc: 0.7449 - val_loss: 0.3744 - val_acc: 0.8336
Epoch 3/100
24/24 [==============================] - 29s 1s/step - loss: 0.3888 - acc: 0.8194 - val_loss: 0.3639 - val_acc: 0.8430
Epoch 4/100
24/24 [==============================] - 29s 1s/step - loss: 0.3237 - acc: 0.8450 - val_loss: 0.2685 - val_acc: 0.8673
Epoch 5/100
24/24 [==============================] - 29s 1s/step - loss: 0.3022 - acc: 0.8698 - val_loss: 0.2496 - val_acc: 0.8860
Epoch 6/100
24/24 [==============================] - 29s 1s/step - loss: 0.2476 - acc: 0.8973 - val_loss: 0.3560 - val_acc: 0.8168
Epoch 7/100
24/24 [==============================] - 29s 1s/step - loss: 0.2658 - acc: 0.8857 - val_loss: 0.2267 - val_acc: 0.9065
Epoch 8/100
24/24 [==============================] - 29s 1s/step - loss: 0.2364 - a

In [15]:
preds

array([ 0.17434557,  0.33020672,  0.01869651, ...,  0.0418326 ,
        0.9970749 ,  0.01072063], dtype=float32)

In [17]:
p = pd.read_csv('sub.csv')
submission = pd.DataFrame()
submission['id']=p['id']
submission['is_iceberg']=preds
submission.to_csv('sub1.csv', index=False)